<a href="https://colab.research.google.com/github/vinu0404/Vi-s-CUT/blob/main/Vi's_CUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import threading
from moviepy.editor import VideoFileClip, concatenate_videoclips
import time

# Function to merge videos
def merge_videos(video1_path, video2_path, output_path, fade_in_duration=1, fade_out_duration=1, crossfade_duration=1, progress_bar=None, status_label=None, button=None):
    try:
        # Disable the button and change its text to show processing
        button.config(state=tk.DISABLED, text="Applying Effects...")

        # Show the "In Progress" label and progress bar
        status_label.grid(row=4, column=1, padx=10, pady=5)
        progress_bar.grid(row=5, column=1, padx=10, pady=10)
        progress_bar.config(mode="indeterminate")
        progress_bar.start()

        # Load the video files
        clip1 = VideoFileClip(video1_path)
        clip2 = VideoFileClip(video2_path)

        time.sleep(1)  # Simulating a delay for visual effect

        # Apply fade-in and fade-out effects
        clip1 = clip1.fadein(fade_in_duration)
        clip2 = clip2.fadeout(fade_out_duration)

        # Merge the two clips with crossfade
        merged_clip = concatenate_videoclips([clip1, clip2.crossfadein(crossfade_duration)], method="compose")

        # Write the output video file
        merged_clip.write_videofile(output_path, codec="libx264", fps=24)

        messagebox.showinfo("Success", f"Video saved at {output_path}")
    except Exception as e:
        messagebox.showerror("Error", f"Failed to process video: {str(e)}")
    finally:
        # Stop the progress bar and reset its mode, then hide it
        progress_bar.stop()
        progress_bar.config(mode="determinate")
        progress_bar['value'] = 0
        progress_bar.grid_forget()  # Hide the progress bar after processing

        # Hide the "In Progress" label
        status_label.grid_forget()

        # Enable the button and reset its text
        button.config(state=tk.NORMAL, text="Merge and Apply Effects")
        clip1.close()
        clip2.close()

# Function to open a file dialog to select a video file
def browse_file(entry_field):
    file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4 *.avi *.mov")])
    entry_field.delete(0, tk.END)
    entry_field.insert(0, file_path)

# Function to open a "Save As" dialog for the output video
def save_file_dialog():
    output_path = filedialog.asksaveasfilename(defaultextension=".mp4", filetypes=[("MP4 files", "*.mp4")])
    return output_path

# Function to process the videos based on the selected action
def process_videos():
    video1_path = video1_entry.get()
    video2_path = video2_entry.get()

    if not video1_path or not video2_path:
        messagebox.showerror("Error", "Please select both video files.")
        return

    # Ask user where to save the output video
    output_path = save_file_dialog()

    if not output_path:
        messagebox.showerror("Error", "Please specify an output file path.")
        return

    try:
        fade_in_duration = int(fade_in_entry.get()) if fade_in_entry.get() else 1
        fade_out_duration = int(fade_out_entry.get()) if fade_out_entry.get() else 1
        crossfade_duration = int(crossfade_entry.get()) if crossfade_entry.get() else 1

        # Start processing in a new thread to avoid freezing the GUI
        threading.Thread(target=merge_videos, args=(video1_path, video2_path, output_path, fade_in_duration, fade_out_duration, crossfade_duration, progress_bar, status_label, process_button)).start()
    except ValueError:
        messagebox.showerror("Error", "Please enter valid durations for fade and crossfade effects.")

# GUI setup
root = tk.Tk()
root.title("Vi's CUT")

# Video 1 selection
video1_label = tk.Label(root, text="Select First Video:")
video1_label.grid(row=0, column=0, padx=10, pady=5)
video1_entry = tk.Entry(root, width=50)
video1_entry.grid(row=0, column=1, padx=10, pady=5)
video1_button = tk.Button(root, text="Browse", command=lambda: browse_file(video1_entry))
video1_button.grid(row=0, column=2, padx=10, pady=5)

# Video 2 selection
video2_label = tk.Label(root, text="Select Second Video:")
video2_label.grid(row=1, column=0, padx=10, pady=5)
video2_entry = tk.Entry(root, width=50)
video2_entry.grid(row=1, column=1, padx=10, pady=5)
video2_button = tk.Button(root, text="Browse", command=lambda: browse_file(video2_entry))
video2_button.grid(row=1, column=2, padx=10, pady=5)

# Fade-in duration input
fade_in_label = tk.Label(root, text="Fade In Duration (seconds):")
fade_in_label.grid(row=2, column=0, padx=10, pady=5)
fade_in_entry = tk.Entry(root, width=10)
fade_in_entry.grid(row=2, column=1, padx=10, pady=5)

# Fade-out duration input
fade_out_label = tk.Label(root, text="Fade Out Duration (seconds):")
fade_out_label.grid(row=3, column=0, padx=10, pady=5)
fade_out_entry = tk.Entry(root, width=10)
fade_out_entry.grid(row=3, column=1, padx=10, pady=5)

# Crossfade duration input
crossfade_label = tk.Label(root, text="Crossfade Duration (seconds):")
crossfade_label.grid(row=4, column=0, padx=10, pady=5)
crossfade_entry = tk.Entry(root, width=10)
crossfade_entry.grid(row=4, column=1, padx=10, pady=5)

# "In Progress" label (initially hidden)
status_label = tk.Label(root, text="In Progress...", fg="blue")
status_label.grid_forget()  # Hidden until needed

# Indeterminate progress bar setup (initially hidden)
progress_bar = ttk.Progressbar(root, mode='indeterminate')
progress_bar.grid_forget()  # Hidden until needed

# Process button
process_button = tk.Button(root, text="Merge and Apply Effects", command=process_videos)
process_button.grid(row=6, column=1, padx=10, pady=10)

root.mainloop()